In [1]:
import spacy
import pandas as pd

import json
import spacy
import requests
import random
import time

import spacy
import tqdm
from collections import Counter

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim

In [2]:
my_api_key="17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W"

In [4]:
def number_of_articles(api_key, keyword, begin_date="20090120", end_date="20240101"):
    url_query = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?fq={keyword}&begin_date={begin_date}&end_date={end_date}&api-key={api_key}"
    print(url_query)
    res = requests.get(url_query)
    json_res = res.json()
    hits = json_res["response"]["meta"]["hits"]
    return hits

print(number_of_articles(api_key=my_api_key, keyword='US and China relations, United States, China, Beijing, Washington'))

https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
684798


In [5]:
def search_nyt_articles(my_api_key, keyword, max_pages, begin_date="20090120", end_date="20240101"):
    docs = []

    for i in range(max_pages):
        api = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={keyword}&begin_date={begin_date}&end_date={end_date}&page={i+1}&api-key={my_api_key}"
        print(api)
        response = requests.get(api)
        data = response.json()
        try:
            docs.extend(data['response']['docs'])
        except KeyError:
            return docs
        time.sleep(12)

    return docs

articles = search_nyt_articles(my_api_key, keyword='US and China relations, United States, China, Beijing, Washington', max_pages=200)

https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=1&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=2&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=3&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=4&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20

https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=40&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=41&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=42&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=43&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_dat

https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=79&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=80&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=81&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=82&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_dat

https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=118&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=119&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=120&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=121&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end

https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=157&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=158&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=159&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=160&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end

https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=196&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=197&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=198&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end_date=20240101&page=199&api-key=17DXHOMUlAY4lhndGsZdGoT6UlsI1N1W
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=US and China relations, United States, China, Beijing, Washington&begin_date=20090120&end

In [6]:
len(articles)

2000

# Research Topic

In the aftermath of Barack Obama's inauguration on January 20, 2009, the initial years of his presidential term were characterized by a prevailing positive sentiment in U.S.-China relations. During this period, media portrayals of China generally leaned toward the favorable, reflecting a climate of cooperation and mutual understanding between the two nations.

However, a palpable shift in diplomatic dynamics occurred with the transition to the Trump administration. The advent of the Trade War and the multifaceted challenges posed by the Covid-19 pandemic introduced a discernible transformation in how China was depicted in the media. This period marked an evolution in U.S.-China relations, portraying a landscape that was not only complex but at times contentious under President Trump's leadership.

Following the conclusion of Trump's presidency, Joe Biden assumed office with a diplomatic approach that aimed to soften relations with China. Despite these efforts, the present geopolitical landscape positions China and the U.S. on opposing sides across various critical issues. The intricate interplay of economic, political, and global challenges has contributed to a complex and multifaceted relationship between the two nations.

Against this backdrop, our research project seeks to undertake a comprehensive exploration of the nuanced evolution of media representations of China-U.S. relations. Our investigation spans from the early years of the Obama administration in 2009 to the present day. 

The choice of The New York Times as a primary source is deliberate, considering its rich history dating back to 1851 and its consistent adherence to high editorial standards, as evidenced by the numerous Pulitzer Prizes it has earned for reporting excellence. Moreover, its reputation as a reliable and influential source positions it as a noteworthy contributor to the discourse on global affairs. The newspaper's comprehensive coverage and global reach provide a broad and diverse perspective, enriching our understanding of the complex dynamics between China and the U.S. The New York Times' adaptability to the digital era, coupled with its significant online presence, ensures that its content is accessible to a vast and diverse global audience. This digital accessibility enhances our ability to capture a wide range of viewpoints and narratives that have shaped perceptions of the China-U.S. relationship over time. Furthermore, by recognizing The New York Times' center-left editorial stance, we affirm our commitment to acknowledge potential bias, while appreciating the newspaper's unique insights. 

In our research, our goal is to provide valuable insights into the reciprocal relationship between media portrayals, particularly from influential sources such as The New York Times, and the dynamic nature of China-U.S. relations. By acknowledging the substantial influence of the newspaper, both within the U.S. and globally, we seek to illuminate its role in shaping public opinion and reflecting prevailing sentiments within society.

In addition, our aim is to capture the diverse narratives and perspectives that have molded our understanding of this pivotal geopolitical relationship over time. Through this exploration, we strive to contribute insights into how media portrayals, especially from influential sources like The New York Times, have not only mirrored but also influenced the evolving nature of China-U.S. relations. This endeavor offers a valuable lens through which to comprehend the intricate complexities of this vital global partnership. Our approach remains thoughtful and inclusive as we navigate the complexities of this global partnership, recognizing the multifaceted nature of media influence. Through this exploration, we aim to provide a thorough and valuable perspective on the evolving narratives surrounding China-U.S. relations, adding depth and context to our research endeavor.

In [12]:
def consolidate_corpus(articles):
    docs = []
    for article in articles:
        lead_paragraph = article.get('lead_paragraph')
        docs.append([article['headline']['main'], lead_paragraph, article['pub_date']])
    group_corpus = pd.DataFrame(docs, columns=['headline', 'lead_paragraph', 'date'])
    return group_corpus

group_corpus = consolidate_corpus(articles)

group_corpus['date'] = pd.to_datetime(group_corpus['date'])

group_corpus = group_corpus.sort_values(by='date')

group_corpus.to_csv('group_corpus_sorted.csv', index=False)

In [14]:
print(group_corpus)

                                               headline  \
568   China Jittery About Obama Amid Signs of Harder...   
890   China Jittery About Obama Amid Signs of Harder...   
536      U.S. Prepares to Broach Hard Issues With China   
299                      Clinton Seeks a Shift on China   
1761       Asia welcomes Clinton, and renewed attention   
...                                                 ...   
350         U.S. Presses China to Stop Flow of Fentanyl   
1696  In Talks With Biden, Xi Seeks to Assure and As...   
1763         Panda Diplomacy Might Not Be Dead Just Yet   
141   For Biden, a Subtle Shift in the Power Balance...   
1638  A Rare Opportunity to See China’s Leader Up Cl...   

                                         lead_paragraph  \
568   WASHINGTON  Whether it was a shot across the ...   
890   WASHINGTON  Whether it was a shot across the ...   
536   WASHINGTON  The Obama administration plans to...   
299   Signaling a new, more vigorous approach to Chi...